In [2]:
from surprise import BaselineOnly
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import PredefinedKFold
from collections import defaultdict
from surprise.model_selection.split import LeaveOneOut
import pandas as pd
import numpy as np
import os

In [3]:
df = pd.read_csv("all_20.csv")
df = df[['user_id','business_id','stars']]

In [3]:
#df = pd.read_csv("all_over_100.csv")

In [4]:
df = df[['user_id','business_id','stars']]

In [1]:
#list(df['business_id'].unique())

In [28]:
len(df.user_id.unique())

4944

In [27]:
len(df.business_id.unique())

2214

In [7]:
df = df.dropna()

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 221262 entries, 0 to 221261
Data columns (total 3 columns):
user_id        221262 non-null object
business_id    221262 non-null object
stars          221262 non-null float64
dtypes: float64(1), object(2)
memory usage: 6.8+ MB


In [9]:
#df_train = pd.read_csv("train_33_selected.csv")

In [10]:
#df_test = pd.read_csv("test_33_selected.csv")

In [11]:
reader = Reader(rating_scale=(1, 5))
#data_train = Dataset.load_from_df(df_train, reader)
#data_test = Dataset.load_from_df(df_test, reader)
data_all = Dataset.load_from_df(df, reader)

In [12]:
df_all = data_all.build_full_trainset()

In [13]:
#df_trainset_train = data_train.build_full_trainset()
#df_trainset_test = df_trainset_train.build_testset()

In [14]:
#df_testset = data_test.build_full_trainset().build_testset()

In [20]:
def get_top_n(predictions, n=10):
    '''Return the top-N recommendation for each user from a set of predictions.

    Args:
        predictions(list of Prediction objects): The list of predictions, as
            returned by the test method of an algorithm.
        n(int): The number of recommendation to output for each user. Default
            is 10.

    Returns:
    A dict where keys are user (raw) ids and values are lists of tuples:
        [(raw item id, rating estimation), ...] of size n.
    '''

    # First map the predictions to each user.
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, est))

    # Then sort the predictions for each user and retrieve the k highest ones.
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]

    return top_n

In [56]:
top_n = defaultdict(list)

In [58]:
for uid, iid, true_r, est_mean, _ in predictions:
    top_n[uid].append((iid, est_mean))

In [59]:
for uid, user_ratings in top_n.items():
    user_ratings.sort(key=lambda x: x[1], reverse=True)
    top_n[uid] = user_ratings[:10]

# Modeling

#### BaselineOnly

In [13]:
from surprise import BaselineOnly

bsl_options = {'method': 'sgd',
               'reg': 0.001,
               'learning_rate': .005,
               }

algo = BaselineOnly(bsl_options=bsl_options)
algo.fit(df_all)

Estimating biases using sgd...


In [14]:
newset = df_all.build_anti_testset()
predictions = algo.test(newset)

In [17]:
pred_base = pd.DataFrame(predictions)

In [25]:
pred_base.head()

,uid,iid,r_ui,est,details
0,tXhscxwFTd-ayLTdPx_2yA,-1m9o3vGRA8IBPNvNqKLmA,3.808747,3.740005,{'was_impossible': False}
1,tXhscxwFTd-ayLTdPx_2yA,-3zffZUHoY8bQjGfPSoBKQ,3.808747,3.560440,{'was_impossible': False}
2,tXhscxwFTd-ayLTdPx_2yA,-8R_-EkGpUhBk55K9Dd4mg,3.808747,2.720285,{'was_impossible': False}
3,tXhscxwFTd-ayLTdPx_2yA,-9YyInW1wapzdNZrhQJ9dg,3.808747,2.136462,{'was_impossible': False}
4,tXhscxwFTd-ayLTdPx_2yA,-AD5PiuJHgdUcAK-Vxao2A,3.808747,2.956387,{'was_impossible': False}


In [26]:
pred_base.to_csv("pred_base.csv", index=False)

In [21]:
top_n = get_top_n(predictions, n=10)

In [22]:
# Print the recommended items for each user
for uid, user_ratings in top_n.items():
    print(uid, [iid for (iid, _) in user_ratings])

AWt-4_RGNqz9u2T8XfOy9g ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ', 'vOMDU31gdylrzBhAKC9QbA']
m2gIW3qe18scLqPw2JP2BQ ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
XqsuWJIwdy0OXYvUX1I2-Q ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
rNjgVKaBh1PlUA6dYB6EUw ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GK

WTE4gtyCs1eJDSCj2Afo_A ['2hSIeCX2cV-chFcBBXrZZA', '9P23-V64kYz3trn9ecaJJA', 'IhNASEZ3XnBHmuuVnWdIwA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'QOE93SXgKd8EsaUT-DaXhA', 'vT812nRctS62PVxUverayw', 'wWuGa3OOQJro_XGe-GKBXA', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
y_IaE3HKJKEsw5Cg62D1RA ['2hSIeCX2cV-chFcBBXrZZA', 'IhNASEZ3XnBHmuuVnWdIwA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
H7fr4A3OlKtDr-sNn8BIlw ['2hSIeCX2cV-chFcBBXrZZA', '9P23-V64kYz3trn9ecaJJA', 'IhNASEZ3XnBHmuuVnWdIwA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
6ZaL6fBlKx5tb8c9opl7rQ ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GK

lIM5d1HsNmdkn9LzV8gKNw ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
hzKQOJI8-7TM-BstJpSVCA ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
ylCz0y57-aVG_Y9rGC6xTA ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
3RTesI_MAwct13LWm4rhLw ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9eca

BdRMVROS1MXOHxr-bdZv0g ['2hSIeCX2cV-chFcBBXrZZA', '9P23-V64kYz3trn9ecaJJA', 'IhNASEZ3XnBHmuuVnWdIwA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'L1-1P3acJc4gEFvWwjXcNQ', 'vOMDU31gdylrzBhAKC9QbA']
0oCX_noDIESiIHKf7Sly1w ['-1m9o3vGRA8IBPNvNqKLmA', '-C8sSrFqaCxp51pyo-fQLQ', '-RJ216TTIghZshCkUlD1WQ', '-iFvYhgysvjkxckCr42NRw', '0q_BHpxbikVtPRRLRu-U0g', '0xvdC8F0HmFpAFkJk6nXLg', '1CaM8eIvl41l4f3V-V-cAw', '1WBVqmo7tgF0z0eGf4fPhQ', '296PZdxSrtH08EUwCsOKMw', '2B46bRpDh49eDyjXGhL_ZQ']
y6UplUH72gl3eBtG6ACa1w ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
FCGCxiXBrj1p_8cscTeQXg ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GK

oKhpMYvIntENZ5UzuoZWUg ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
O5VAQ-dbR2Bk0OZLL1L0Ig ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
-R_hd3xKlnUIi_m-IZsY9Q ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GKBXA', '9P23-V64kYz3trn9ecaJJA', 'vT812nRctS62PVxUverayw', 'tMdgY7Ou4k3bYZZulJCiuQ', 'O19VReN1I2TBrJsbXUAIJg', 'L1-1P3acJc4gEFvWwjXcNQ']
eYNLZkBYHTgGUcmgTVOWBQ ['2hSIeCX2cV-chFcBBXrZZA', 'O7UMzd3i-Zk8dMeyY9ZwoA', 'IhNASEZ3XnBHmuuVnWdIwA', 'QOE93SXgKd8EsaUT-DaXhA', 'wWuGa3OOQJro_XGe-GK

In [50]:
from surprise import BaselineOnly

bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 8,
               'reg_i': 15
               }

algo_als = BaselineOnly(bsl_options=bsl_options)
algo_als.fit(df_trainset_train)

train_pred = algo_als.test(df_trainset_test)
print("BaselineOnly train biased RMSE", accuracy.rmse(train_pred))
test_pred = algo_als.test(df_testset)
print("BaselineOnly test unbiased RMSE", accuracy.rmse(test_pred))

Estimating biases using als...
RMSE: 1.0665
BaselineOnly train biased RMSE 1.0665105660993361
RMSE: 1.2108
BaselineOnly test unbiased RMSE 1.2107896707675672


#### KNN

In [29]:
from surprise import KNNBaseline
algo_knn = KNNBaseline(k=40, user_based=True)
algo_knn.fit(df_all)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


In [30]:
predictions_knn = algo_knn.test(newset)

In [31]:
pred_knn = pd.DataFrame(predictions_knn)

In [64]:
pred_knn.to_csv("pred_knn.csv", index=False)

In [65]:
pred_knn.head()

,uid,iid,r_ui,est,details
0,tXhscxwFTd-ayLTdPx_2yA,-1m9o3vGRA8IBPNvNqKLmA,3.808747,3.795506,"{'actual_k': 26, 'was_impossible': False}"
1,tXhscxwFTd-ayLTdPx_2yA,-3zffZUHoY8bQjGfPSoBKQ,3.808747,3.420844,"{'actual_k': 40, 'was_impossible': False}"
2,tXhscxwFTd-ayLTdPx_2yA,-8R_-EkGpUhBk55K9Dd4mg,3.808747,2.797524,"{'actual_k': 16, 'was_impossible': False}"
3,tXhscxwFTd-ayLTdPx_2yA,-9YyInW1wapzdNZrhQJ9dg,3.808747,2.120093,"{'actual_k': 23, 'was_impossible': False}"
4,tXhscxwFTd-ayLTdPx_2yA,-AD5PiuJHgdUcAK-Vxao2A,3.808747,2.930751,"{'actual_k': 40, 'was_impossible': False}"


In [63]:
pred_knn.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10740076 entries, 0 to 10740075
Data columns (total 5 columns):
uid        object
iid        object
r_ui       float64
est        float64
details    object
dtypes: float64(2), object(3)
memory usage: 409.7+ MB


In [ ]:
train_pred_knn = algo_knn.test(df_trainset_test)
print("KNN train biased RMSE", accuracy.rmse(train_pred_knn))
test_pred_knn = algo_knn.test(df_testset)
print("KNN test unbiased RMSE", accuracy.rmse(test_pred_knn))

In [ ]:
#cross_validate(algo_knn, data_train, measures=['RMSE', 'MAE'], cv=5, verbose=True)

#### PMF

In [38]:
from surprise import SVD
algo_pmf = SVD(n_factors=12, n_epochs=20, biased=False)
algo_pmf.fit(df_trainset_train)

train_pred_pmf = algo_pmf.test(df_trainset_test)
print("PMF train biased RMSE", accuracy.rmse(train_pred_pmf))
test_pred_pmf = algo_pmf.test(df_testset)
print("PMF test unbiased RMSE", accuracy.rmse(test_pred_pmf))

RMSE: 0.9462
PMF train biased RMSE 0.9462048849220677
RMSE: 1.5357
PMF test unbiased RMSE 1.5356683095858437


In [39]:
cross_validate(algo_pmf, data_train, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3664  1.3728  1.3646  1.3655  1.3725  1.3684  0.0036  
MAE (testset)     1.0549  1.0607  1.0534  1.0550  1.0594  1.0567  0.0028  
Fit time          15.57   15.76   15.75   15.73   15.74   15.71   0.07    
Test time         1.30    1.28    0.89    0.86    0.85    1.04    0.21    


{'fit_time': (15.566722869873047,
  15.762358903884888,
  15.749221086502075,
  15.731513023376465,
  15.742002964019775),
 'test_mae': array([1.05493793, 1.06067975, 1.05337667, 1.05501464, 1.05939546]),
 'test_rmse': array([1.36638166, 1.37284885, 1.36459553, 1.3655026 , 1.3724832 ]),
 'test_time': (1.299198865890503,
  1.2790918350219727,
  0.8946449756622314,
  0.8617708683013916,
  0.8522119522094727)}

In [26]:
from surprise import SVD
algo_pmf = SVD(biased=False)

for trainset, testset in loo.split(data_all):
    
    algo_pmf.fit(trainset)
    predictions = algo_pmf.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 1.5246
RMSE: 1.5259
RMSE: 1.5192
RMSE: 1.5190
RMSE: 1.5192


#### SVD

In [35]:
from surprise import SVD
algo_svd_1 = SVD(n_factors=20, n_epochs=15)
algo_svd_1.fit(df_all)

In [36]:
predictions_svd_1 = algo_svd_1.test(newset)

In [37]:
pred_svd_1 = pd.DataFrame(predictions_svd_1)

In [38]:
pred_svd_1.to_csv("pred_svd_1.csv", index=False)

In [34]:
train_pred_svd_1 = algo_svd_1.test(df_trainset_test)
print("SVD train biased RMSE", accuracy.rmse(train_pred_svd_1))
test_pred_svd_1 = algo_svd_1.test(df_testset)
print("SVD test unbiased RMSE", accuracy.rmse(test_pred_svd_1))

RMSE: 1.0249
SVD train biased RMSE 1.024856211461352
RMSE: 1.2118
SVD test unbiased RMSE 1.2117590546524613


In [23]:
from surprise import SVD
algo_svd_2 = SVD(n_factors=50, n_epochs=10)
algo_svd_2.fit(df_trainset_train)

train_pred_svd_2 = algo_svd_2.test(df_trainset_test)
print("SVD train biased RMSE", accuracy.rmse(train_pred_svd_2))
test_pred_svd_2 = algo_svd_2.test(df_testset)
print("SVD test unbiased RMSE", accuracy.rmse(test_pred_svd_2))

RMSE: 1.0388
SVD train biased RMSE 1.0387571058602665
RMSE: 1.2199
SVD test unbiased RMSE 1.2199389887459124


In [28]:
from surprise import SVD
algo_svd_3 = SVD(n_factors=30, n_epochs=10)
algo_svd_3.fit(df_trainset_train)

train_pred_svd_3 = algo_svd_3.test(df_trainset_test)
print("SVD train biased RMSE", accuracy.rmse(train_pred_svd_3))
test_pred_svd_3 = algo_svd_3.test(df_testset)
print("SVD test unbiased RMSE", accuracy.rmse(test_pred_svd_3))

RMSE: 1.0608
SVD train biased RMSE 1.0608437830508364
RMSE: 1.2187
SVD test unbiased RMSE 1.2186500127989746


In [30]:
from surprise import SVD
algo_svd_4 = SVD(n_factors=30, n_epochs=15)
algo_svd_4.fit(df_trainset_train)

train_pred_svd_4 = algo_svd_4.test(df_trainset_test)
print("SVD train biased RMSE", accuracy.rmse(train_pred_svd_4))
test_pred_svd_4 = algo_svd_4.test(df_testset)
print("SVD test unbiased RMSE", accuracy.rmse(test_pred_svd_4))

RMSE: 0.9611
SVD train biased RMSE 0.9610708138026886
RMSE: 1.2151
SVD test unbiased RMSE 1.2150919274110736


In [35]:
cross_validate(algo_svd_1, data_train, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1482  1.1487  1.1477  1.1494  1.1472  1.1482  0.0008  
MAE (testset)     0.9133  0.9121  0.9130  0.9142  0.9124  0.9130  0.0007  
Fit time          14.69   16.53   15.14   14.76   15.44   15.31   0.67    
Test time         1.25    1.12    1.15    1.12    1.12    1.15    0.05    


{'fit_time': (14.690735816955566,
  16.53309202194214,
  15.13542103767395,
  14.755640983581543,
  15.436825037002563),
 'test_mae': array([0.91330799, 0.91208184, 0.91299981, 0.91418933, 0.91238658]),
 'test_rmse': array([1.14818215, 1.1487161 , 1.14769818, 1.14935646, 1.14718293]),
 'test_time': (1.246934175491333,
  1.125,
  1.1499409675598145,
  1.116776943206787,
  1.117764949798584)}

In [ ]:
algo_svd = SVD()

for trainset, testset in loo.split(data_all):
    
    algo_svd.fit(trainset)
    predictions = algo_svd.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

#### SVDpp

In [56]:
from surprise import SVDpp
algo_svdpp_1 = SVDpp(n_factors=10, n_epochs=10)
algo_svdpp_1.fit(df_trainset_train)

train_pred_svdpp_1 = algo_svdpp_1.test(df_trainset_test)
print("SVDpp train biased RMSE", accuracy.rmse(train_pred_svdpp_1))
test_pred_svdpp_1 = algo_svdpp_1.test(df_testset)
print("SVDpp test unbiased RMSE", accuracy.rmse(test_pred_svdpp_1))

RMSE: 1.0514
SVDpp train biased RMSE 1.051354400669383
RMSE: 1.2118
SVDpp test unbiased RMSE 1.2117677323392348


In [58]:
from surprise import SVDpp
algo_svdpp_2 = SVDpp(n_factors=5, n_epochs=5)
algo_svdpp_2.fit(df_trainset_train)

train_pred_svdpp_2 = algo_svdpp_2.test(df_trainset_test)
print("SVDpp train biased RMSE", accuracy.rmse(train_pred_svdpp_2))
test_pred_svdpp_2 = algo_svdpp_2.test(df_testset)
print("SVDpp test unbiased RMSE", accuracy.rmse(test_pred_svdpp_2))

RMSE: 1.1077
SVDpp train biased RMSE 1.1076843252153576
RMSE: 1.2254
SVDpp test unbiased RMSE 1.2254464718146858


In [60]:
from surprise import SVDpp
algo_svdpp = SVDpp()
algo_svdpp.fit(df_trainset_train)

In [61]:
train_pred_svdpp = algo_svdpp.test(df_trainset_test)
print("SVDpp train biased RMSE", accuracy.rmse(train_pred_svdpp))
test_pred_svdpp = algo_svdpp.test(df_testset)
print("SVDpp test unbiased RMSE", accuracy.rmse(test_pred_svdpp))

RMSE: 0.8178
SVDpp train biased RMSE 0.8177596675789183
RMSE: 1.2224
SVDpp test unbiased RMSE 1.222432032057506


In [59]:
cross_validate(algo_svdpp_1, data_train, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVDpp on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1457  1.1449  1.1483  1.1488  1.1483  1.1472  0.0016  
MAE (testset)     0.9121  0.9110  0.9132  0.9139  0.9136  0.9128  0.0011  
Fit time          137.76  154.74  164.27  160.81  159.68  155.45  9.36    
Test time         6.79    8.01    7.91    10.56   7.72    8.20    1.26    


{'fit_time': (137.75702691078186,
  154.73514199256897,
  164.268061876297,
  160.80781507492065,
  159.68211698532104),
 'test_mae': array([0.91207069, 0.91103457, 0.91324639, 0.91389616, 0.91358074]),
 'test_rmse': array([1.14566271, 1.14489693, 1.14832405, 1.14878058, 1.1483142 ]),
 'test_time': (6.7922258377075195,
  8.010616779327393,
  7.907413005828857,
  10.5589120388031,
  7.715319871902466)}

In [30]:
from surprise import SVDpp

algo_svdpp = SVDpp()

for trainset, testset in loo.split(data_all):
    
    algo_svdpp.fit(trainset)
    predictions = algo_svdpp.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 1.2270


KeyboardInterrupt: 

#### NMF

In [54]:
from surprise import NMF

algo_nmf_1 = NMF(n_factors=500,n_epochs=10,biased=True)
algo_nmf_1.fit(df_trainset_train)

train_pred_nmf_1 = algo_nmf_1.test(df_trainset_test)
print("NMF train biased RMSE", accuracy.rmse(train_pred_nmf_1))
test_pred_nmf_1 = algo_nmf_1.test(df_testset)
print("NMF test unbiased RMSE", accuracy.rmse(test_pred_nmf_1))

RMSE: 1.8887
NMF train biased RMSE 1.8886954324179221
RMSE: 2.0244
NMF test unbiased RMSE 2.024350988002533


In [53]:
from surprise import NMF

algo_nmf_2 = NMF(n_factors=3,n_epochs=5,biased=True)
algo_nmf_2.fit(df_trainset_train)

train_pred_nmf_2 = algo_nmf_2.test(df_trainset_test)
print("NMF train biased RMSE", accuracy.rmse(train_pred_nmf_2))
test_pred_nmf_2 = algo_nmf_2.test(df_testset)
print("NMF test unbiased RMSE", accuracy.rmse(test_pred_nmf_2))

RMSE: 1.0423
NMF train biased RMSE 1.0422730231469668
RMSE: 1.2333
NMF test unbiased RMSE 1.2332936708221804


In [39]:
from surprise import NMF

algo_nmf_3 = NMF(n_factors=2,n_epochs=5,biased=True)
algo_nmf_3.fit(df_all)

In [40]:
predictions_nmf_3 = algo_nmf_3.test(newset)

In [41]:
pred_nmf_3 = pd.DataFrame(predictions_nmf_3)

In [42]:
pred_nmf_3.to_csv("pred_nmf_3.csv",index=False)

In [43]:
pred_nmf_3.head()

,uid,iid,r_ui,est,details
0,tXhscxwFTd-ayLTdPx_2yA,-1m9o3vGRA8IBPNvNqKLmA,3.808747,3.694761,{'was_impossible': False}
1,tXhscxwFTd-ayLTdPx_2yA,-3zffZUHoY8bQjGfPSoBKQ,3.808747,3.732986,{'was_impossible': False}
2,tXhscxwFTd-ayLTdPx_2yA,-8R_-EkGpUhBk55K9Dd4mg,3.808747,3.185755,{'was_impossible': False}
3,tXhscxwFTd-ayLTdPx_2yA,-9YyInW1wapzdNZrhQJ9dg,3.808747,2.763986,{'was_impossible': False}
4,tXhscxwFTd-ayLTdPx_2yA,-AD5PiuJHgdUcAK-Vxao2A,3.808747,3.151796,{'was_impossible': False}


In [ ]:
train_pred_nmf_3 = algo_nmf_3.test(df_trainset_test)
print("NMF train biased RMSE", accuracy.rmse(train_pred_nmf_3))
test_pred_nmf_3 = algo_nmf_3.test(df_testset)
print("NMF test unbiased RMSE", accuracy.rmse(test_pred_nmf_3))

In [58]:
from surprise import NMF

algo_nmf = NMF()
algo_nmf.fit(df_trainset_train)

In [59]:
train_pred_nmf = algo_nmf.test(df_trainset_test)
print("NMF train biased RMSE", accuracy.rmse(train_pred_nmf))
test_pred_nmf = algo_nmf.test(df_testset)
print("NMF test unbiased RMSE", accuracy.rmse(test_pred_nmf))

RMSE: 0.6135
NMF train biased RMSE 0.6135013087584554
RMSE: 1.3930
NMF test unbiased RMSE 1.3929995422178636


In [55]:
cross_validate(algo_nmf_2, data_train, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm NMF on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.1725  1.1729  1.1715  1.1734  1.1682  1.1717  0.0018  
MAE (testset)     0.9237  0.9249  0.9243  0.9249  0.9206  0.9237  0.0016  
Fit time          4.38    6.95    8.13    6.42    10.26   7.23    1.94    
Test time         1.48    1.75    1.68    2.13    1.73    1.75    0.21    


{'fit_time': (4.378977060317993,
  6.947607040405273,
  8.1286461353302,
  6.4235310554504395,
  10.257351160049438),
 'test_mae': array([0.92372323, 0.92493659, 0.92429811, 0.92490481, 0.92059365]),
 'test_rmse': array([1.17247172, 1.17287832, 1.17151232, 1.17341319, 1.16824885]),
 'test_time': (1.4765958786010742,
  1.7464051246643066,
  1.6839540004730225,
  2.1256210803985596,
  1.7344229221343994)}

In [29]:
from surprise import NMF

algo_nmf = NMF()

for trainset, testset in loo.split(data_all):
    
    algo_nmf.fit(trainset)
    predictions = algo_nmf.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 1.3859
RMSE: 1.3855
RMSE: 1.3893
RMSE: 1.3844
RMSE: 1.3880


#### CoClustering

In [44]:
from surprise import CoClustering

algo_clu_2 = CoClustering(n_cltr_u=3, n_cltr_i=2, n_epochs=5)
algo_clu_2.fit(df_trainset_train)

train_pred_clu_2 = algo_clu_2.test(df_trainset_test)
print("CoClustering train biased RMSE", accuracy.rmse(train_pred_clu_2))
test_pred_clu_2 = algo_clu_2.test(df_testset)
print("CoClustering test unbiased RMSE", accuracy.rmse(test_pred_clu_2))

RMSE: 0.9696
CoClustering train biased RMSE 0.969627115190611
RMSE: 1.3367
CoClustering test unbiased RMSE 1.3367309507289586


In [45]:
from surprise import CoClustering

algo_clu_1 = CoClustering(n_cltr_u=2, n_cltr_i=2, n_epochs=5)
algo_clu_1.fit(df_trainset_train)

train_pred_clu_1 = algo_clu_1.test(df_trainset_test)
print("CoClustering train biased RMSE", accuracy.rmse(train_pred_clu_1))
test_pred_clu_1 = algo_clu_1.test(df_testset)
print("CoClustering test unbiased RMSE", accuracy.rmse(test_pred_clu_1))

RMSE: 0.9823
CoClustering train biased RMSE 0.9822603928968215
RMSE: 1.3331
CoClustering test unbiased RMSE 1.3330527088008828


In [44]:
from surprise import CoClustering

algo_clu_3 = CoClustering(n_cltr_u=2, n_cltr_i=3, n_epochs=10)
algo_clu_3.fit(df_all)

In [45]:
predictions_clu_3 = algo_clu_3.test(newset)

In [46]:
pred_clu_3 = pd.DataFrame(predictions_clu_3)

In [47]:
pred_clu_3.to_csv("pred_clu_3.csv",index=False)

In [48]:
pred_clu_3.head()

,uid,iid,r_ui,est,details
0,tXhscxwFTd-ayLTdPx_2yA,-1m9o3vGRA8IBPNvNqKLmA,3.808747,4.062707,{'was_impossible': False}
1,tXhscxwFTd-ayLTdPx_2yA,-3zffZUHoY8bQjGfPSoBKQ,3.808747,3.636673,{'was_impossible': False}
2,tXhscxwFTd-ayLTdPx_2yA,-8R_-EkGpUhBk55K9Dd4mg,3.808747,2.826993,{'was_impossible': False}
3,tXhscxwFTd-ayLTdPx_2yA,-9YyInW1wapzdNZrhQJ9dg,3.808747,2.133336,{'was_impossible': False}
4,tXhscxwFTd-ayLTdPx_2yA,-AD5PiuJHgdUcAK-Vxao2A,3.808747,3.043402,{'was_impossible': False}


In [ ]:
train_pred_clu_3 = algo_clu_3.test(df_trainset_test)
print("CoClustering train biased RMSE", accuracy.rmse(train_pred_clu_3))
test_pred_clu_3 = algo_clu_3.test(df_testset)
print("CoClustering test unbiased RMSE", accuracy.rmse(test_pred_clu_3))

In [55]:
from surprise import CoClustering

algo_clu = CoClustering()
algo_clu.fit(df_trainset_train)

In [56]:
train_pred_clu = algo_clu.test(df_trainset_test)
print("CoClustering train biased RMSE", accuracy.rmse(train_pred_clu))
test_pred_clu = algo_clu.test(df_testset)
print("CoClustering test unbiased RMSE", accuracy.rmse(test_pred_clu))

RMSE: 0.9677
CoClustering train biased RMSE 0.9677336945010659
RMSE: 1.3402
CoClustering test unbiased RMSE 1.3401557418150414


In [47]:
cross_validate(algo_clu_3, data_train, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2743  1.2712  1.2698  1.2734  1.2712  1.2720  0.0016  
MAE (testset)     0.9498  0.9486  0.9482  0.9507  0.9490  0.9492  0.0009  
Fit time          15.92   15.75   21.61   15.73   13.82   16.57   2.64    
Test time         3.08    1.01    2.13    0.90    0.89    1.60    0.88    


{'fit_time': (15.916110038757324,
  15.751596212387085,
  21.609081983566284,
  15.73489499092102,
  13.816364049911499),
 'test_mae': array([0.94981774, 0.9485647 , 0.9481505 , 0.95067871, 0.94895787]),
 'test_rmse': array([1.27425011, 1.27124372, 1.26976499, 1.27342349, 1.27121491]),
 'test_time': (3.0831899642944336,
  1.0070009231567383,
  2.1267809867858887,
  0.8989031314849854,
  0.8862819671630859)}

In [28]:
from surprise import CoClustering

algo_clu = CoClustering()

for trainset, testset in loo.split(data_all):
    
    algo_clu.fit(trainset)
    predictions = algo_clu.test(testset)

    # Compute and print Root Mean Squared Error
    accuracy.rmse(predictions, verbose=True)

RMSE: 1.3439
RMSE: 1.3483
RMSE: 1.3454
RMSE: 1.3469
RMSE: 1.3460


#### Ensemble

In [49]:
test = pd.DataFrame(pred_base)

In [50]:
test['est_cluster'] = pd.DataFrame(pred_clu_3)['est']
test['est_svd'] = pd.DataFrame(pred_svd_1)['est']
#test['est_svdpp'] = pd.DataFrame(pred_svdpp_1)['est']
test['est_nmf'] = pd.DataFrame(pred_nmf_3)['est']
test['est_knn'] = pd.DataFrame(pred_knn)['est']

In [51]:
test['est_mean'] = test[['est','est_cluster','est_svd','est_knn','est_nmf']].mean(axis=1)

In [53]:
test.to_csv("pred_20_est.csv",index=False)

In [54]:
test.head()

,uid,iid,r_ui,est,details,est_cluster,est_svd,est_nmf,est_knn,est_mean
0,tXhscxwFTd-ayLTdPx_2yA,-1m9o3vGRA8IBPNvNqKLmA,3.808747,3.740005,{'was_impossible': False},4.062707,3.758913,3.694761,3.795506,3.810378
1,tXhscxwFTd-ayLTdPx_2yA,-3zffZUHoY8bQjGfPSoBKQ,3.808747,3.560440,{'was_impossible': False},3.636673,3.555759,3.732986,3.420844,3.581340
2,tXhscxwFTd-ayLTdPx_2yA,-8R_-EkGpUhBk55K9Dd4mg,3.808747,2.720285,{'was_impossible': False},2.826993,2.857589,3.185755,2.797524,2.877629
3,tXhscxwFTd-ayLTdPx_2yA,-9YyInW1wapzdNZrhQJ9dg,3.808747,2.136462,{'was_impossible': False},2.133336,2.357052,2.763986,2.120093,2.302186
4,tXhscxwFTd-ayLTdPx_2yA,-AD5PiuJHgdUcAK-Vxao2A,3.808747,2.956387,{'was_impossible': False},3.043402,3.057378,3.151796,2.930751,3.027943


In [55]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10740076 entries, 0 to 10740075
Data columns (total 10 columns):
uid            object
iid            object
r_ui           float64
est            float64
details        object
est_cluster    float64
est_svd        float64
est_nmf        float64
est_knn        float64
est_mean       float64
dtypes: float64(7), object(3)
memory usage: 819.4+ MB


In [195]:
test = pd.read_csv("pred_10_est.csv")

In [196]:
pred_mean = test[['uid','iid','est_mean']]
pred_mean.head()

,uid,iid,est_mean
0,n_HMBasYWRQxVjts_QyF9w,-0BxAGlIk5DJAGVkpqBXxg,3.861474
1,n_HMBasYWRQxVjts_QyF9w,-0RkJ_uIduNLWQrphbADRw,3.870671
2,n_HMBasYWRQxVjts_QyF9w,-1m9o3vGRA8IBPNvNqKLmA,4.548343
3,n_HMBasYWRQxVjts_QyF9w,-1vfRrlnNnNJ5boOVghMPA,3.597368
4,n_HMBasYWRQxVjts_QyF9w,-3zffZUHoY8bQjGfPSoBKQ,4.421059


In [226]:
top_10 = pred_mean.groupby('uid').apply(lambda x: x.sort_values(['est_mean'],ascending=False).head(10))

In [227]:
top_10_reset = top_10.reset_index(drop=True)

In [232]:
bs = pd.read_csv("selected_business_all_info.csv")
bs.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,"1775 E Tropicana Ave, Ste 29","{'BusinessAcceptsCreditCards': 'True', 'BYOBCo...",PZ-LZzSlhSe9utkQYU8pFg,"Restaurants, Italian",Las Vegas,NaN,0,36.100016,-115.128528,Carluccio's Tivoli Gardens,89119.0,40,4.0,NV
1,6055 E Lake Mead Blvd,"{'BusinessAcceptsCreditCards': 'True', 'HasTV'...",tstimHoMcYbkSC4eBA1wEg,"Mexican, Restaurants, Patisserie/Cake Shop, Fo...",Las Vegas,"{'Monday': '11:0-21:0', 'Wednesday': '10:0-21:...",1,36.195615,-115.040529,Maria's Mexican Restaurant & Bakery,89156.0,184,4.5,NV
2,6125 Spring Mountain Rd,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",kANF0dbeoW34s2vwh6Umfw,"Fast Food, Food, Restaurants, Ice Cream & Froz...",Las Vegas,NaN,0,36.125031,-115.225620,Dairy Queen,89146.0,33,2.0,NV
3,4343 N Rancho Dr,NaN,X8mtoSxY8whtmbDlj0D3Aw,"Restaurants, Chinese",Las Vegas,NaN,1,36.238596,-115.233331,Imperial Asian Buffet,89030.0,4,2.0,NV
4,"Artisan Hotel, 1501 W Sahara Ave","{'BusinessAcceptsCreditCards': 'True', 'Restau...",bJP4l_BGq2CudEu0m-wNjg,"Restaurants, Pizza, Italian, American (New)",Las Vegas,"{'Monday': '16:0-0:0', 'Wednesday': '16:0-0:0'...",0,36.143672,-115.169792,Artisan Fine Dining Room,89102.0,3,2.0,NV


In [256]:
top = top_10_reset[['iid']][:10]
out_put = pd.merge(top,
               bs[['name','address','categories','business_id']],
               left_on='iid',
               right_on='business_id',
               how='left')

out_put[['name','address','categories']]

,name,address,categories
0,Brew Tea Bar,"7380 S Rainbow Blvd, Ste 101","Tea Rooms, Desserts, Cafes, Restaurants, Food,..."
1,In-N-Out Burger,10040 W Sahara Ave,"Restaurants, Burgers, Fast Food"
2,Lip Smacking Foodie Tours,3635 Las Vegas Blvd S,"Hotels, Food, Event Planning & Services, Hotel..."
3,Art of Flavors,"1616 S Las Vegas Blvd, Ste 130","Gelato, Ice Cream & Frozen Yogurt, Restaurants..."
4,Mordeo Boutique Wine Bar,"5420 W Spring Mountain Rd, Ste 108","Tapas/Small Plates, Wine Bars, Spanish, Americ..."
5,POTs,"1745 S Rainbow Blvd, Ste A","Vegan, Egyptian, Restaurants, Vegetarian, Midd..."
6,EDO Gastro Tapas & Wine,"3400 S Jones Blvd, Ste 11A","Spanish, Tapas/Small Plates, Restaurants"
7,Garden Grill,302 S Rampart Blvd,"Restaurants, Farmers Market, Street Vendors, M..."
8,Kame Omakase,"7331 W Lake Mead Blvd, Ste 104","Seafood, Japanese, Restaurants, Sushi Bars"
9,Poke Heaven,4747 S Maryland Pkwy,"Poke, Korean, Asian Fusion, Japanese, Hawaiian..."


In [246]:
a = bs[bs['business_id'].isin(top)][['name','address','categories']].to_html()

In [247]:
geo_10 = pd.merge(geo_9,
               bs[['name','latitude','longitude','business_id']],
               left_on='top_10',
               right_on='business_id',
               how='left')

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>name</th>\n      <th>address</th>\n      <th>categories</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>350</th>\n      <td>Art of Flavors</td>\n      <td>1616 S Las Vegas Blvd, Ste 130</td>\n      <td>Gelato, Ice Cream &amp; Frozen Yogurt, Restaurants...</td>\n    </tr>\n    <tr>\n      <th>422</th>\n      <td>In-N-Out Burger</td>\n      <td>10040 W Sahara Ave</td>\n      <td>Restaurants, Burgers, Fast Food</td>\n    </tr>\n    <tr>\n      <th>779</th>\n      <td>POTs</td>\n      <td>1745 S Rainbow Blvd, Ste A</td>\n      <td>Vegan, Egyptian, Restaurants, Vegetarian, Midd...</td>\n    </tr>\n    <tr>\n      <th>1411</th>\n      <td>Mordeo Boutique Wine Bar</td>\n      <td>5420 W Spring Mountain Rd, Ste 108</td>\n      <td>Tapas/Small Plates, Wine Bars, Spanish, Americ...</td>\n    </tr>\n    <tr>\n      <th>2952</th>\n      <td>Kame Omakase</td>\n      <td>7331

In [200]:
n = [1,2,3,4,5,6,7,8,9,10]*int(len(top_10_reset)/10)
p = pd.DataFrame(n)
top_10_reset['top'] = p[0]

In [201]:
top_10_tidy = pd.merge(top_10_reset,
               bs[['name','latitude','longitude','business_id']],
               left_on='iid',
               right_on='business_id',
               how='left')

In [202]:
top_10_v = top_10_tidy.loc[:, :'longitude']

In [203]:
top_10_v.to_csv("top_10_v_100k.csv",index=False)

In [204]:
top_10_v.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158010 entries, 0 to 158009
Data columns (total 7 columns):
uid          158010 non-null object
iid          158010 non-null object
est_mean     158010 non-null float64
top          158010 non-null int64
name         158010 non-null object
latitude     158010 non-null float64
longitude    158010 non-null float64
dtypes: float64(3), int64(1), object(3)
memory usage: 9.6+ MB


In [205]:
df_t = top_10_reset.pivot(index='uid',columns='top',values='iid')

In [206]:
df_t.columns.name = None
df_t_r = df_t.reset_index()

In [207]:
df_t_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15801 entries, 0 to 15800
Data columns (total 11 columns):
uid    15801 non-null object
1      15801 non-null object
2      15801 non-null object
3      15801 non-null object
4      15801 non-null object
5      15801 non-null object
6      15801 non-null object
7      15801 non-null object
8      15801 non-null object
9      15801 non-null object
10     15801 non-null object
dtypes: object(11)
memory usage: 1.3+ MB


In [131]:
#df_t_r.to_csv("top10_4944.csv",index=False)

In [134]:
bs = pd.read_csv("selected_business_all_info.csv")

In [140]:
bs.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,postal_code,review_count,stars,state
0,"1775 E Tropicana Ave, Ste 29","{'BusinessAcceptsCreditCards': 'True', 'BYOBCo...",PZ-LZzSlhSe9utkQYU8pFg,"Restaurants, Italian",Las Vegas,NaN,0,36.100016,-115.128528,Carluccio's Tivoli Gardens,89119.0,40,4.0,NV
1,6055 E Lake Mead Blvd,"{'BusinessAcceptsCreditCards': 'True', 'HasTV'...",tstimHoMcYbkSC4eBA1wEg,"Mexican, Restaurants, Patisserie/Cake Shop, Fo...",Las Vegas,"{'Monday': '11:0-21:0', 'Wednesday': '10:0-21:...",1,36.195615,-115.040529,Maria's Mexican Restaurant & Bakery,89156.0,184,4.5,NV
2,6125 Spring Mountain Rd,"{'BusinessAcceptsCreditCards': 'True', 'Restau...",kANF0dbeoW34s2vwh6Umfw,"Fast Food, Food, Restaurants, Ice Cream & Froz...",Las Vegas,NaN,0,36.125031,-115.225620,Dairy Queen,89146.0,33,2.0,NV
3,4343 N Rancho Dr,NaN,X8mtoSxY8whtmbDlj0D3Aw,"Restaurants, Chinese",Las Vegas,NaN,1,36.238596,-115.233331,Imperial Asian Buffet,89030.0,4,2.0,NV
4,"Artisan Hotel, 1501 W Sahara Ave","{'BusinessAcceptsCreditCards': 'True', 'Restau...",bJP4l_BGq2CudEu0m-wNjg,"Restaurants, Pizza, Italian, American (New)",Las Vegas,"{'Monday': '16:0-0:0', 'Wednesday': '16:0-0:0'...",0,36.143672,-115.169792,Artisan Fine Dining Room,89102.0,3,2.0,NV


In [208]:
geo = pd.merge(df_t_r,
               bs[['name','latitude','longitude','business_id']],
               left_on=1,
               right_on='business_id',
               how='left')


In [209]:
geo.columns = ['uid','top_1','top_2','top_3','top_4','top_5','top_6','top_7','top_8','top_9','top_10',
              'top_1_name','top_1_lat','top_1_long','o']


In [210]:
geo_1 = geo[['uid','top_1','top_2','top_3','top_4','top_5','top_6','top_7','top_8','top_9','top_10','top_1_name','top_1_lat','top_1_long']]
geo_1.head()

,uid,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10,top_1_name,top_1_lat,top_1_long
0,---1lKK3aKOuomHnwAkAow,IhNASEZ3XnBHmuuVnWdIwA,aUCJt0FluWAxkOd8s7TkWQ,iBPyahdJRP5y0t25fF2W9w,O7UMzd3i-Zk8dMeyY9ZwoA,vT812nRctS62PVxUverayw,Jjwa-WCK73efAwVZRksWSw,7hWNnAj4VwK6FAUBN8E8lg,k9b96JXlNewj36ddrce9Jw,9P23-V64kYz3trn9ecaJJA,cOqd9ou3uf-1k3OWFo-_CQ,Brew Tea Bar,36.054227,-115.242407
1,--2vR0DIsmQ6WfcSzKWigw,O7UMzd3i-Zk8dMeyY9ZwoA,9P23-V64kYz3trn9ecaJJA,2hSIeCX2cV-chFcBBXrZZA,k9b96JXlNewj36ddrce9Jw,aUCJt0FluWAxkOd8s7TkWQ,IhNASEZ3XnBHmuuVnWdIwA,xxAGaqYnboeR0SilS7DbAg,HY1qcwlWLkH2_4dIWjCmQQ,QOE93SXgKd8EsaUT-DaXhA,vT812nRctS62PVxUverayw,Art of Flavors,36.151555,-115.152610
2,--RlSfc-QmcHFGHyX6aVjA,O7UMzd3i-Zk8dMeyY9ZwoA,QOE93SXgKd8EsaUT-DaXhA,AN0bWhisCf6LN9eHZ7DQ3w,9P23-V64kYz3trn9ecaJJA,2hSIeCX2cV-chFcBBXrZZA,k9b96JXlNewj36ddrce9Jw,IhNASEZ3XnBHmuuVnWdIwA,aUCJt0FluWAxkOd8s7TkWQ,FZQ3O3sxICy1HaJLhPbGYA,xxAGaqYnboeR0SilS7DbAg,Art of Flavors,36.151555,-115.152610
3,--WLHsm-AC4jcol2gOkmCQ,QOE93SXgKd8EsaUT-DaXhA,IhNASEZ3XnBHmuuVnWdIwA,O7UMzd3i-Zk8dMeyY9ZwoA,9P23-V64kYz3trn9ecaJJA,FZQ3O3sxICy1HaJLhPbGYA,AN0bWhisCf6LN9eHZ7DQ3w,k9b96JXlNewj36ddrce9Jw,xxAGaqYnboeR0SilS7DbAg,2hSIeCX2cV-chFcBBXrZZA,hIUKufhwR6Ifn7bi0-phLA,Snow Ono Shave Ice,36.169710,-115.123695
4,--ZNfWKj1VyVElRx6-g1fg,aUCJt0FluWAxkOd8s7TkWQ,QOE93SXgKd8EsaUT-DaXhA,2hSIeCX2cV-chFcBBXrZZA,9P23-V64kYz3trn9ecaJJA,IhNASEZ3XnBHmuuVnWdIwA,O7UMzd3i-Zk8dMeyY9ZwoA,AN0bWhisCf6LN9eHZ7DQ3w,-CQokjildrY7UZezXCdEBw,kafUXRNfQm3e_U14S2WbmA,iBPyahdJRP5y0t25fF2W9w,In-N-Out Burger,36.144807,-115.313467


In [211]:
geo_2 = pd.merge(geo_1,
               bs[['name','latitude','longitude','business_id']],
               left_on='top_2',
               right_on='business_id',
               how='left')

In [212]:
geo_2.head()

,uid,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,top_10,top_1_name,top_1_lat,top_1_long,name,latitude,longitude,business_id
0,---1lKK3aKOuomHnwAkAow,IhNASEZ3XnBHmuuVnWdIwA,aUCJt0FluWAxkOd8s7TkWQ,iBPyahdJRP5y0t25fF2W9w,O7UMzd3i-Zk8dMeyY9ZwoA,vT812nRctS62PVxUverayw,Jjwa-WCK73efAwVZRksWSw,7hWNnAj4VwK6FAUBN8E8lg,k9b96JXlNewj36ddrce9Jw,9P23-V64kYz3trn9ecaJJA,cOqd9ou3uf-1k3OWFo-_CQ,Brew Tea Bar,36.054227,-115.242407,In-N-Out Burger,36.144807,-115.313467,aUCJt0FluWAxkOd8s7TkWQ
1,--2vR0DIsmQ6WfcSzKWigw,O7UMzd3i-Zk8dMeyY9ZwoA,9P23-V64kYz3trn9ecaJJA,2hSIeCX2cV-chFcBBXrZZA,k9b96JXlNewj36ddrce9Jw,aUCJt0FluWAxkOd8s7TkWQ,IhNASEZ3XnBHmuuVnWdIwA,xxAGaqYnboeR0SilS7DbAg,HY1qcwlWLkH2_4dIWjCmQQ,QOE93SXgKd8EsaUT-DaXhA,vT812nRctS62PVxUverayw,Art of Flavors,36.151555,-115.152610,Kame Omakase,36.194769,-115.252472,9P23-V64kYz3trn9ecaJJA
2,--RlSfc-QmcHFGHyX6aVjA,O7UMzd3i-Zk8dMeyY9ZwoA,QOE93SXgKd8EsaUT-DaXhA,AN0bWhisCf6LN9eHZ7DQ3w,9P23-V64kYz3trn9ecaJJA,2hSIeCX2cV-chFcBBXrZZA,k9b96JXlNewj36ddrce9Jw,IhNASEZ3XnBHmuuVnWdIwA,aUCJt0FluWAxkOd8s7TkWQ,FZQ3O3sxICy1HaJLhPbGYA,xxAGaqYnboeR0SilS7DbAg,Art of Flavors,36.151555,-115.152610,Snow Ono Shave Ice,36.169710,-115.123695,QOE93SXgKd8EsaUT-DaXhA
3,--WLHsm-AC4jcol2gOkmCQ,QOE93SXgKd8EsaUT-DaXhA,IhNASEZ3XnBHmuuVnWdIwA,O7UMzd3i-Zk8dMeyY9ZwoA,9P23-V64kYz3trn9ecaJJA,FZQ3O3sxICy1HaJLhPbGYA,AN0bWhisCf6LN9eHZ7DQ3w,k9b96JXlNewj36ddrce9Jw,xxAGaqYnboeR0SilS7DbAg,2hSIeCX2cV-chFcBBXrZZA,hIUKufhwR6Ifn7bi0-phLA,Snow Ono Shave Ice,36.169710,-115.123695,Brew Tea Bar,36.054227,-115.242407,IhNASEZ3XnBHmuuVnWdIwA
4,--ZNfWKj1VyVElRx6-g1fg,aUCJt0FluWAxkOd8s7TkWQ,QOE93SXgKd8EsaUT-DaXhA,2hSIeCX2cV-chFcBBXrZZA,9P23-V64kYz3trn9ecaJJA,IhNASEZ3XnBHmuuVnWdIwA,O7UMzd3i-Zk8dMeyY9ZwoA,AN0bWhisCf6LN9eHZ7DQ3w,-CQokjildrY7UZezXCdEBw,kafUXRNfQm3e_U14S2WbmA,iBPyahdJRP5y0t25fF2W9w,In-N-Out Burger,36.144807,-115.313467,Snow Ono Shave Ice,36.169710,-115.123695,QOE93SXgKd8EsaUT-DaXhA


In [213]:
geo_3 = pd.merge(geo_2,
               bs[['name','latitude','longitude','business_id']],
               left_on='top_3',
               right_on='business_id',
               how='left')

In [214]:
geo_4 = pd.merge(geo_3,
               bs[['name','latitude','longitude','business_id']],
               left_on='top_4',
               right_on='business_id',
               how='left')

In [215]:
geo_5 = pd.merge(geo_4,
               bs[['name','latitude','longitude','business_id']],
               left_on='top_5',
               right_on='business_id',
               how='left')

In [216]:
geo_6 = pd.merge(geo_5,
               bs[['name','latitude','longitude','business_id']],
               left_on='top_6',
               right_on='business_id',
               how='left')

In [217]:
geo_7 = pd.merge(geo_6,
               bs[['name','latitude','longitude','business_id']],
               left_on='top_7',
               right_on='business_id',
               how='left')

In [218]:
geo_8 = pd.merge(geo_7,
               bs[['name','latitude','longitude','business_id']],
               left_on='top_8',
               right_on='business_id',
               how='left')

In [219]:
geo_9 = pd.merge(geo_8,
               bs[['name','latitude','longitude','business_id']],
               left_on='top_9',
               right_on='business_id',
               how='left')

In [220]:
geo_10 = pd.merge(geo_9,
               bs[['name','latitude','longitude','business_id']],
               left_on='top_10',
               right_on='business_id',
               how='left')

In [221]:
geo_10.columns.tolist()

['uid',
 'top_1',
 'top_2',
 'top_3',
 'top_4',
 'top_5',
 'top_6',
 'top_7',
 'top_8',
 'top_9',
 'top_10',
 'top_1_name',
 'top_1_lat',
 'top_1_long',
 'name_x',
 'latitude_x',
 'longitude_x',
 'business_id_x',
 'name_y',
 'latitude_y',
 'longitude_y',
 'business_id_y',
 'name_x',
 'latitude_x',
 'longitude_x',
 'business_id_x',
 'name_y',
 'latitude_y',
 'longitude_y',
 'business_id_y',
 'name_x',
 'latitude_x',
 'longitude_x',
 'business_id_x',
 'name_y',
 'latitude_y',
 'longitude_y',
 'business_id_y',
 'name_x',
 'latitude_x',
 'longitude_x',
 'business_id_x',
 'name_y',
 'latitude_y',
 'longitude_y',
 'business_id_y',
 'name',
 'latitude',
 'longitude',
 'business_id']

In [222]:
geo_10.columns = ['uid',
 'top_1',
 'top_2',
 'top_3',
 'top_4',
 'top_5',
 'top_6',
 'top_7',
 'top_8',
 'top_9',
 'top_10',
 'top_1_name',
 'top_1_lat',
 'top_1_long',
 'top_2_name',
 'top_2_latitude',
 'top_2_longitude',
 'business_id_x',
 'top_3_name',
 'top_3_latitude',
 'top_3_longitude',
 'business_id_y',
 'top_4_name',
 'top_4_latitude',
 'top_4_longitude',
 'business_id_x',
 'top_5_name',
 'top_5_latitude',
 'top_5_longitude',
 'business_id_y',
 'top_6_name',
 'top_6_latitude',
 'top_6_longitude',
 'business_id_x',
 'top_7_name',
 'top_7_latitude',
 'top_7_longitude',
 'business_id_y',
 'top_8_name',
 'top_8_latitude',
 'top_8_longitude',
 'business_id_x',
 'top_9_name',
 'top_9_latitude',
 'top_9_longitude',
 'business_id_y',
 'top_10_name',
 'top_10_latitude',
 'top_10_longitude',
 'business_id']

In [223]:
top_10_h = geo_10[['uid',
 'top_1',
 'top_2',
 'top_3',
 'top_4',
 'top_5',
 'top_6',
 'top_7',
 'top_8',
 'top_9',
 'top_10',
 'top_1_name',
 'top_1_lat',
 'top_1_long',
 'top_2_name',
 'top_2_latitude',
 'top_2_longitude',
 'top_3_name',
 'top_3_latitude',
 'top_3_longitude',
 'top_4_name',
 'top_4_latitude',
 'top_4_longitude',
 'top_5_name',
 'top_5_latitude',
 'top_5_longitude',
 'top_6_name',
 'top_6_latitude',
 'top_6_longitude',
 'top_7_name',
 'top_7_latitude',
 'top_7_longitude',
 'top_8_name',
 'top_8_latitude',
 'top_8_longitude',
 'top_9_name',
 'top_9_latitude',
 'top_9_longitude',
 'top_10_name',
 'top_10_latitude',
 'top_10_longitude']]

In [224]:
top_10_h.to_csv("top_10_h_100k.csv",index=False)

In [225]:
top_10_h.head()

,uid,top_1,top_2,top_3,top_4,top_5,top_6,top_7,top_8,top_9,...,top_7_longitude,top_8_name,top_8_latitude,top_8_longitude,top_9_name,top_9_latitude,top_9_longitude,top_10_name,top_10_latitude,top_10_longitude
0,---1lKK3aKOuomHnwAkAow,IhNASEZ3XnBHmuuVnWdIwA,aUCJt0FluWAxkOd8s7TkWQ,iBPyahdJRP5y0t25fF2W9w,O7UMzd3i-Zk8dMeyY9ZwoA,vT812nRctS62PVxUverayw,Jjwa-WCK73efAwVZRksWSw,7hWNnAj4VwK6FAUBN8E8lg,k9b96JXlNewj36ddrce9Jw,9P23-V64kYz3trn9ecaJJA,...,-115.224069,Garden Grill,36.166783,-115.286197,Kame Omakase,36.194769,-115.252472,Poke Heaven,36.104162,-115.137631
1,--2vR0DIsmQ6WfcSzKWigw,O7UMzd3i-Zk8dMeyY9ZwoA,9P23-V64kYz3trn9ecaJJA,2hSIeCX2cV-chFcBBXrZZA,k9b96JXlNewj36ddrce9Jw,aUCJt0FluWAxkOd8s7TkWQ,IhNASEZ3XnBHmuuVnWdIwA,xxAGaqYnboeR0SilS7DbAg,HY1qcwlWLkH2_4dIWjCmQQ,QOE93SXgKd8EsaUT-DaXhA,...,-115.141365,D E Thai Kitchen,36.158441,-115.151200,Snow Ono Shave Ice,36.169710,-115.123695,Mordeo Boutique Wine Bar,36.125917,-115.215268
2,--RlSfc-QmcHFGHyX6aVjA,O7UMzd3i-Zk8dMeyY9ZwoA,QOE93SXgKd8EsaUT-DaXhA,AN0bWhisCf6LN9eHZ7DQ3w,9P23-V64kYz3trn9ecaJJA,2hSIeCX2cV-chFcBBXrZZA,k9b96JXlNewj36ddrce9Jw,IhNASEZ3XnBHmuuVnWdIwA,aUCJt0FluWAxkOd8s7TkWQ,FZQ3O3sxICy1HaJLhPbGYA,...,-115.242407,In-N-Out Burger,36.144807,-115.313467,Bon Breads Baking Company,36.069600,-115.172258,Stripchezze Food Truck,36.179668,-115.141365
3,--WLHsm-AC4jcol2gOkmCQ,QOE93SXgKd8EsaUT-DaXhA,IhNASEZ3XnBHmuuVnWdIwA,O7UMzd3i-Zk8dMeyY9ZwoA,9P23-V64kYz3trn9ecaJJA,FZQ3O3sxICy1HaJLhPbGYA,AN0bWhisCf6LN9eHZ7DQ3w,k9b96JXlNewj36ddrce9Jw,xxAGaqYnboeR0SilS7DbAg,2hSIeCX2cV-chFcBBXrZZA,...,-115.286197,Stripchezze Food Truck,36.179668,-115.141365,Pressed For Juice,36.075943,-115.298066,Baguette Cafe,36.066865,-115.273241
4,--ZNfWKj1VyVElRx6-g1fg,aUCJt0FluWAxkOd8s7TkWQ,QOE93SXgKd8EsaUT-DaXhA,2hSIeCX2cV-chFcBBXrZZA,9P23-V64kYz3trn9ecaJJA,IhNASEZ3XnBHmuuVnWdIwA,O7UMzd3i-Zk8dMeyY9ZwoA,AN0bWhisCf6LN9eHZ7DQ3w,-CQokjildrY7UZezXCdEBw,kafUXRNfQm3e_U14S2WbmA,...,-115.092483,Toddy Shop,36.171423,-115.245246,Pizzeria Monzú,36.115110,-115.223600,Lip Smacking Foodie Tours,36.114537,-115.172678


In [64]:
train = pd.DataFrame(train_pred)

In [67]:
train.to_csv("train_est.csv")

In [65]:
train['est_cluster'] = pd.DataFrame(train_pred_clu_3)['est']
train['est_svd'] = pd.DataFrame(train_pred_svd_1)['est']
train['est_svdpp'] = pd.DataFrame(train_pred_svdpp_1)['est']
train['est_nmf'] = pd.DataFrame(train_pred_nmf_2)['est']
#train['est_knn'] = pd.DataFrame(train_pred_knn)['est']

In [66]:
train['est_mean'] = train[['est','est_cluster','est_svd','est_svdpp','est_nmf']].mean(axis=1)

In [69]:
from sklearn.metrics import mean_squared_error
from math import sqrt

train_rmse = sqrt(mean_squared_error(train['r_ui'], train['est_mean']))
test_rmse = sqrt(mean_squared_error(test['r_ui'], test['est_mean']))

In [70]:
print("ensemble train rmse", train_rmse)
print("ensemble test rmse", test_rmse)

ensemble train rmse 1.0115591563326694
ensemble test rmse 1.2078211477272622


### Blending

In [3]:
from sklearn.metrics import mean_squared_error
from math import sqrt

def train_test_model_performance(clf, X_train, y_train, X_test, y_test):
    # Fit a model by providing X and y from training set
    clf.fit(X_train, y_train)

    # Make prediction on the training data
    y_train_pred = clf.predict(X_train)

    # Make predictions on test data
    y_test_pred = clf.predict(X_test)
    
    train_rmse = sqrt(mean_squared_error(y_train, y_train_pred))
    test_rmse = sqrt(mean_squared_error(y_test, y_test_pred))

    return train_rmse, test_rmse

In [83]:
#X_train = train[['est','est_cluster','est_svd','est_svdpp','est_nmf']]
#y_train = train['r_ui']
from sklearn.model_selection import train_test_split

X = test[['est','est_cluster','est_svd','est_svdpp','est_nmf']]
y = test['r_ui']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=77)

In [84]:
from sklearn.linear_model import LinearRegression

reg = LinearRegression()

train_rmse, test_rmse = train_test_model_performance(reg, X_train, y_train, X_test, y_test)

print('train rmse', train_rmse)
print('test rmse', test_rmse)

train rmse 1.2040537215312836
test rmse 1.2046226769340245


In [87]:
from sklearn.linear_model import Ridge

rig = Ridge(alpha=10)

train_rmse, test_rmse = train_test_model_performance(rig, X_train, y_train, X_test, y_test)

print('train rmse', train_rmse)
print('test rmse', test_rmse)

train rmse 1.2040540469888974
test rmse 1.2046244824045316


In [86]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=20, max_depth=7, min_samples_leaf=10, max_features=3)

train_rmse, test_rmse = train_test_model_performance(rf, X_train, y_train, X_test, y_test)

print('train rmse', train_rmse)
print('test rmse', test_rmse)

train rmse 1.1945114072313194
test rmse 1.2022756054693744


#### LightGBM

In [41]:
import datetime
import lightgbm as lgb
import numpy as np
import os
import pandas as pd
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import haversine

ImportError: No module named 'haversine'